In [6]:
import pandas as pd
import numpy as np

In [2]:
target = pd.read_csv('done/target_whole.csv',index_col=0)

In [22]:
target.columns

Index(['user', 'time_stamp', 'adgroup_id', 'pid', 'clk', 'seq_len', 'cate_id',
       'campaign_id', 'customer', 'brand', 'price', 'from_purchase', 'cate'],
      dtype='object')

In [23]:
df = target[['user','campaign_id','from_purchase']]

In [24]:
# Campaign 등장 횟수 집계 함수 (sequence 를 기준으로)
# 한 seq 에서 등장 1 , 등장 x 0 으로 집계 

def count_campaigns_around_labels(df, label_column='from_purchase', id_column='campaign_id'):
    counts = {}
    for user, user_df in df.groupby('user'):
        label_indices = user_df.index[user_df[label_column] == 1].tolist()
        
        # Label 1 (== sequence 구분 기준) 인덱스를 기준으로 분할하여 집계
        segments = np.split(user_df, [i + 1 for i in label_indices])
        
        for segment in segments:
            unique_ads = segment[id_column].unique()
            for ad in unique_ads:
                if ad in counts:
                    counts[ad] += 1
                else:
                    counts[ad] = 1
    
    return counts

campaign_counts_around_labels = count_campaigns_around_labels(df)
campaign_counts_around_labels_df = pd.DataFrame(list(campaign_counts_around_labels.items()), columns=['campaign_id', 'count'])


In [28]:
# 구매 자체 row 는 campaign 이 없어서 nan 값으로 나옴 : 제거 
campaigns = campaign_counts_around_labels_df.dropna()

In [31]:
# 빈도 상위 1천개 뽑아 리스트 화 
campaign_valid = campaigns.sort_values(by='count',ascending=False).head(1000)['campaign_id'].to_list()

- 상위 1천개 campaign 으로만 이루어진 user ID 몇개 있는지 체크
    -  how? : campaign filtering 이전과 이후 각 ID의 Seq_len 가 동일하면됨

In [71]:
# seq_before_filtered 정의 
# filtering 되지 않은 데이터의 seq len 
seq_before_filtered = target[target['from_purchase'] == 0].groupby('user')['user'].agg(seq_len = 'count').reset_index()

In [72]:
# seq_filtered 정의 

# 전체 log 에서 구매 여부 지움 (sequence 를 구성할 때 필요한 노출 Log 만 남김)
df_1 =  target[target['from_purchase'] == 0] 

# 그 중 위에서 정의한 campaign_valid 에 포함되지 않는 노출은 모두 지움 
df_2 = df_1[df_1['campaign_id'].isin(campaign_valid)]

# 그 후 seq_len 를 측정 
seq_filtered = df_2.groupby('user')['user'].agg(seq_len = 'count').reset_index()

# 이렇게 하면, valid 한 campaign ID 로만 구성된 user seq 는 seq_len 가 filtering 되기 전후가 똑같을 것
# invalid 한 campaign ID 가 존재하면 seq_len 가 줄어들 것 
# invalid 한 campaign ID 만 존재하면 해당 user ID 가 모두 사라질 것 

In [73]:
# filtering 되지 않은 데이터프레임의 user ID 를 seq_filtered 에 맞추어 필터링 

seq_before_filtered = seq_before_filtered[seq_before_filtered['user'].isin(seq_filtered.user.to_list())]

In [74]:
seq_before_filtered = seq_before_filtered.reset_index()
seq_filtered = seq_filtered.reset_index()

In [76]:
seq_before_filtered['filtered_seq_len'] = seq_filtered['seq_len']

In [81]:
# 필터링 전후가 똑같은 user 만 list 화 
valid_samples = seq_before_filtered[seq_before_filtered['seq_len'] == seq_before_filtered['filtered_seq_len']]['user'].to_list()

In [84]:
## save it 
# pd.DataFrame(valid_samples).to_csv('done/sample_experiment/valid_samples.csv',index=False)